In [1]:
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.singleton_classifier import SingletonClassifierModelBuilder
from functools import reduce

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
data = get_markable_dataframe("data/training/markables.csv", word_vector, idx_by_word)

data.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,previous_words,next_words,is_singleton
0,1,[1],0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,[],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 2]","[0.0, 1.0]"
1,2,"[4, 5]",0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[1, 2, 3]","[6, 7, 8, 9, 10, 2, 11, 12, 13, 14]","[0.0, 1.0]"
2,3,"[6, 7, 8]",0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]",1,0,"[1, 2, 3, 4, 5]","[9, 10, 2, 11, 12, 13, 14, 15, 16, 17]","[1.0, 0.0]"
3,4,[10],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","[2, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[0.0, 1.0]"
4,5,"[12, 13]",0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[3, 4, 5, 6, 7, 8, 9, 10, 2, 11]","[14, 15, 16, 17, 18, 19, 4, 5, 20, 21]","[0.0, 1.0]"


In [4]:
max_text_length = 10
max_prev_words_length = 10
max_next_words_length = 10

data_text = pad_sequences(data.text, maxlen=max_text_length, padding='post')
data_previous_words = pad_sequences(data.previous_words.map(lambda seq: seq[(-1*max_prev_words_length):]), maxlen=max_prev_words_length, padding='pre')
data_next_words = pad_sequences(data.next_words.map(lambda seq: seq[:max_next_words_length]), maxlen=max_next_words_length, padding='post')
data_syntactic = data[['is_pronoun', 'entity_type', 'is_proper_name', 'is_first_person']]

data_syntactic = np.array(list(map(lambda p: reduce(lambda x,y: x + y, [i if type(i) is list else [i] for i in p]), data_syntactic.values)))
label = np.vstack(data.is_singleton)

# Build Model

## Words

In [5]:
words_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [6]:
words_model = words_model_builder.create_model()

In [7]:
words_model.fit([data_text], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.2877 - acc: 0.8774
Epoch 2/20
5032/5032 [==============================] - 5s 937us/step - loss: 0.1854 - acc: 0.9171
Epoch 3/20
5032/5032 [==============================] - 5s 1ms/step - loss: 0.1481 - acc: 0.9298
Epoch 4/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1360 - acc: 0.9362
Epoch 5/20
5032/5032 [==============================] - 5s 958us/step - loss: 0.1209 - acc: 0.9408
Epoch 6/20
5032/5032 [==============================] - 5s 1ms/step - loss: 0.1253 - acc: 0.9420
Epoch 7/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1127 - acc: 0.9404
Epoch 8/20
5032/5032 [==============================] - 5s 1ms/step - loss: 0.1397 - acc: 0.9344
Epoch 9/20
5032/5032 [==============================] - 5s 1ms/step - loss: 0.1108 - acc: 0.9448
Epoch 10/20
5032/5032 [==============================] - 5s 1ms/step - loss: 0.1051 - acc: 0.9459
Epoch 11/20
5032/5032 [==

In [8]:
words_model.save('models/singleton_classifiers/words.model')

## Context

In [9]:
context_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [10]:
context_model = context_model_builder.create_model()

In [11]:
context_model.fit([data_previous_words, data_next_words], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 10s 2ms/step - loss: 0.4636 - acc: 0.8309
Epoch 2/20
5032/5032 [==============================] - 7s 1ms/step - loss: 0.3546 - acc: 0.8601
Epoch 3/20
5032/5032 [==============================] - 8s 1ms/step - loss: 0.2402 - acc: 0.9155
Epoch 4/20
5032/5032 [==============================] - 8s 2ms/step - loss: 0.1375 - acc: 0.9501
Epoch 5/20
5032/5032 [==============================] - 8s 2ms/step - loss: 0.0884 - acc: 0.9696
Epoch 6/20
5032/5032 [==============================] - 8s 2ms/step - loss: 0.0810 - acc: 0.9762
Epoch 7/20
5032/5032 [==============================] - 8s 2ms/step - loss: 0.0442 - acc: 0.9867
Epoch 8/20
5032/5032 [==============================] - 8s 2ms/step - loss: 0.0471 - acc: 0.9869
Epoch 9/20
5032/5032 [==============================] - 8s 2ms/step - loss: 0.0441 - acc: 0.9879
Epoch 10/20
5032/5032 [==============================] - 10s 2ms/step - loss: 0.0378 - acc: 0.9887
Epoch 11/20
5032/5032 [====

In [12]:
context_model.save('models/singleton_classifiers/context.model')

## Syntactic

In [13]:
syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    syntactic_features_num=data_syntactic.shape[1]
)

In [14]:
syntactic_model = syntactic_model_builder.create_model()

In [15]:
syntactic_model.fit([data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.3115 - acc: 0.8370
Epoch 2/20
5032/5032 [==============================] - 3s 653us/step - loss: 0.2672 - acc: 0.8396
Epoch 3/20
5032/5032 [==============================] - 3s 665us/step - loss: 0.2617 - acc: 0.8514
Epoch 4/20
5032/5032 [==============================] - 3s 684us/step - loss: 0.2633 - acc: 0.8605
Epoch 5/20
5032/5032 [==============================] - 3s 683us/step - loss: 0.2623 - acc: 0.8527
Epoch 6/20
5032/5032 [==============================] - 5s 906us/step - loss: 0.2557 - acc: 0.8579
Epoch 7/20
5032/5032 [==============================] - 3s 649us/step - loss: 0.2609 - acc: 0.8567
Epoch 8/20
5032/5032 [==============================] - 3s 691us/step - loss: 0.2533 - acc: 0.8589
Epoch 9/20
5032/5032 [==============================] - 4s 838us/step - loss: 0.2530 - acc: 0.8615
Epoch 10/20
5032/5032 [==============================] - 4s 730us/step - loss: 0.2516 - acc: 0.8623
Epoch 11/20

In [16]:
syntactic_model.save('models/singleton_classifiers/syntactic.model')

## Words + Context

In [17]:
words_context_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    embedding_matrix=embedding_matrix
)

In [18]:
words_context_model = words_context_model_builder.create_model()

In [19]:
words_context_model.fit([data_text, data_previous_words, data_next_words], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 13s 2ms/step - loss: 0.3456 - acc: 0.8519
Epoch 2/20
5032/5032 [==============================] - 10s 2ms/step - loss: 0.1690 - acc: 0.9356
Epoch 3/20
5032/5032 [==============================] - 11s 2ms/step - loss: 0.0745 - acc: 0.9710
Epoch 4/20
5032/5032 [==============================] - 11s 2ms/step - loss: 0.0515 - acc: 0.9851
Epoch 5/20
5032/5032 [==============================] - 11s 2ms/step - loss: 0.0323 - acc: 0.9863
Epoch 6/20
5032/5032 [==============================] - 11s 2ms/step - loss: 0.0228 - acc: 0.9934
Epoch 7/20
5032/5032 [==============================] - 11s 2ms/step - loss: 0.0280 - acc: 0.9928
Epoch 8/20
5032/5032 [==============================] - 12s 2ms/step - loss: 0.0428 - acc: 0.9903
Epoch 9/20
5032/5032 [==============================] - 11s 2ms/step - loss: 0.0206 - acc: 0.9928
Epoch 10/20
5032/5032 [==============================] - 11s 2ms/step - loss: 0.0241 - acc: 0.9948
Epoch 11/20
5032/50

In [20]:
words_context_model.save('models/singleton_classifiers/words_context.model')

## Words + Syntactic

In [21]:
words_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [22]:
words_syntactic_model = words_syntactic_model_builder.create_model()

In [23]:
words_syntactic_model.fit([data_text, data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 9s 2ms/step - loss: 0.2551 - acc: 0.8768
Epoch 2/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1654 - acc: 0.9223
Epoch 3/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1409 - acc: 0.9306
Epoch 4/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1306 - acc: 0.9394
Epoch 5/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1246 - acc: 0.9398
Epoch 6/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1115 - acc: 0.9440
Epoch 7/20
5032/5032 [==============================] - 7s 1ms/step - loss: 0.1184 - acc: 0.9390
Epoch 8/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1078 - acc: 0.9416
Epoch 9/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.1009 - acc: 0.9475
Epoch 10/20
5032/5032 [==============================] - 6s 1ms/step - loss: 0.0998 - acc: 0.9438
Epoch 11/20
5032/5032 [======

In [24]:
words_syntactic_model.save('models/singleton_classifiers/words_syntactic.model')

## Context + Syntactic

In [25]:
context_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [26]:
context_syntactic_model = context_syntactic_model_builder.create_model()

In [27]:
context_syntactic_model.fit([data_previous_words, data_next_words, data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 13s 3ms/step - loss: 0.3720 - acc: 0.8343
Epoch 2/20
5032/5032 [==============================] - 10s 2ms/step - loss: 0.2294 - acc: 0.8909
Epoch 3/20
5032/5032 [==============================] - 12s 2ms/step - loss: 0.1367 - acc: 0.9479
Epoch 4/20
5032/5032 [==============================] - 9s 2ms/step - loss: 0.0821 - acc: 0.9762
Epoch 5/20
5032/5032 [==============================] - 10s 2ms/step - loss: 0.0574 - acc: 0.9833
Epoch 6/20
5032/5032 [==============================] - 10s 2ms/step - loss: 0.0678 - acc: 0.9811
Epoch 7/20
5032/5032 [==============================] - 10s 2ms/step - loss: 0.0390 - acc: 0.9885
Epoch 8/20
5032/5032 [==============================] - 12s 2ms/step - loss: 0.0206 - acc: 0.9932
Epoch 9/20
5032/5032 [==============================] - 10s 2ms/step - loss: 0.0283 - acc: 0.9924
Epoch 10/20
5032/5032 [==============================] - 9s 2ms/step - loss: 0.0161 - acc: 0.9940
Epoch 11/20
5032/5032

In [28]:
context_syntactic_model.save('models/singleton_classifiers/context_syntactic.model')

## Words + Context + Syntactic

In [29]:
words_context_syntactic_model_builder = SingletonClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=data_syntactic.shape[1]
)

In [30]:
words_context_syntactic_model = words_context_syntactic_model_builder.create_model()

In [31]:
words_context_syntactic_model.fit([data_text, data_previous_words, data_next_words, data_syntactic], label, epochs=20)

Epoch 1/20
5032/5032 [==============================] - 18s 4ms/step - loss: 0.3064 - acc: 0.8716
Epoch 2/20
5032/5032 [==============================] - 14s 3ms/step - loss: 0.1589 - acc: 0.9457
Epoch 3/20
5032/5032 [==============================] - 14s 3ms/step - loss: 0.0879 - acc: 0.9728
Epoch 4/20
5032/5032 [==============================] - 15s 3ms/step - loss: 0.0521 - acc: 0.9859
Epoch 5/20
5032/5032 [==============================] - 15s 3ms/step - loss: 0.0417 - acc: 0.9895
Epoch 6/20
5032/5032 [==============================] - 17s 3ms/step - loss: 0.0262 - acc: 0.9930
Epoch 7/20
5032/5032 [==============================] - 16s 3ms/step - loss: 0.0230 - acc: 0.9924
Epoch 8/20
5032/5032 [==============================] - 15s 3ms/step - loss: 0.0267 - acc: 0.9940
Epoch 9/20
5032/5032 [==============================] - 15s 3ms/step - loss: 0.0248 - acc: 0.9926
Epoch 10/20
5032/5032 [==============================] - 13s 3ms/step - loss: 0.0138 - acc: 0.9972
Epoch 11/20
5032/50

In [32]:
words_context_syntactic_model.save('models/singleton_classifiers/words_context_syntactic.model')